In [1]:
import visual_behavior.data_access.loading as loading
import visual_behavior.data_access.utilities as data_utilities
import visual_behavior.validation.sdk as sdk_validation
from visual_behavior.validation.sdk import ValidateSDK
from visual_behavior import database as db

import datetime
import pandas as pd

%widescreen
%standard_imports

applied a custom magic command to make full use of screen width
will only work if command is defined locally
replace with the following to replicate functionality: 
	from IPython.core.display import display, HTML
	display(HTML("<style>.container { width:100% !important; }</style>")
imported:
	os
	pandas as pd
	numpy as np
	matplotlib.pyplot as plt


In [2]:
def get_donor_from_specimen_id(specimen_id):
    res = db.lims_query('select * from specimens where id = {}'.format(specimen_id))
    if len(res['donor_id']) == 1:
        return res['donor_id'].iloc[0]
    elif len(res['donor_id']) == 0:
        return None
    elif len(res['donor_id']) > 1:
        print('found more than one donor ID for specimen ID {}'.format(specimen_id))
        return res['donor_id'].iloc[0]

cache = loading.get_visual_behavior_cache()
behavior_session_table = cache.get_behavior_session_table().reset_index()
filtered_ophys_experiment_table = loading.get_filtered_ophys_experiment_table()

# get donor id for experiment_table
filtered_ophys_experiment_table['donor_id'] = filtered_ophys_experiment_table['specimen_id'].map(
    lambda sid: get_donor_from_specimen_id(sid)
)

# get behavior donor dataframe - all mice in behavior table
behavior_donors = pd.DataFrame({'donor_id':behavior_session_table['donor_id'].unique()})
# add a flag identifying which donors have associated ophys sessions
behavior_donors['donor_in_ophys'] = behavior_donors['donor_id'].map(
    lambda did: did in list(filtered_ophys_experiment_table['donor_id'].unique())
)

# merge back in behavior donors to determine which behavior sessions have associated ophys
behavior_session_table = behavior_session_table.merge(
    behavior_donors,
    left_on='donor_id',
    right_on='donor_id',
    how='left',
)

# get project table
project_table = db.lims_query("select id,code from projects")
query = '''SELECT behavior_sessions.id, specimens.project_id FROM specimens
JOIN donors ON specimens.donor_id=donors.id
JOIN behavior_sessions ON donors.id=behavior_sessions.donor_id'''
behavior_id_project_id_map = db.lims_query(query).rename(columns={'id':'behavior_session_id'}).merge(
    project_table,
    left_on='project_id',
    right_on='id',
    how='left',
).drop(columns=['id']).rename(columns={'code':'project_code'}).drop_duplicates('behavior_session_id').set_index('behavior_session_id')

# merge project table with behavior sessions
behavior_session_table = behavior_session_table.merge(
    behavior_id_project_id_map.reset_index(),
    left_on='behavior_session_id',
    right_on='behavior_session_id',
    how='left'
)

# add a boolean for whether or not a session is in the filtered experiment table
def osid_in_filtered_experiments(osid):
    if pd.notnull(osid):
        return osid in filtered_ophys_experiment_table['ophys_session_id'].unique()
    else:
        return True
behavior_session_table['in_filtered_experiments'] = behavior_session_table['ophys_session_id'].apply(osid_in_filtered_experiments)

# add missing session types (I have no idea why some are missing!)
def get_session_type(osid):
    if osid in filtered_ophys_experiment_table['ophys_session_id'].unique().tolist():
        return filtered_ophys_experiment_table.query('ophys_session_id == {}'.format(osid)).iloc[0]['session_type']
    else:
        return None

# drop sessions for mice with no ophys AND sessions that aren't in the filtered list
behavior_session_table = behavior_session_table.set_index('behavior_session_id').query('donor_in_ophys and in_filtered_experiments').copy()
for idx,row in behavior_session_table.iterrows():
    if pd.isnull(row['session_type']):
        behavior_session_table.at[idx, 'session_type'] = get_session_type(row['ophys_session_id'])

validation_results = sdk_validation.get_validation_results().sort_index()
behavior_session_table = behavior_session_table.merge(
    validation_results,
    left_index=True,
    right_index=True,
    how='left'
)

Getting behavior-only session data. This might take a while...


In [3]:
sort_by = ['is_ophys','project_code','session_type','behavior_session_id']

behavior_session_table.sort_values(by=sort_by).to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020.11.03_sdk_attribute_validation.csv', index=False)


In [11]:
import pandas as pd
sdk_validation_path = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020.11.03_sdk_attribute_validation.csv'
sdk_validation_df = pd.read_csv(sdk_validation_path)
metadata_failures = sdk_validation_df.query('metadata == False and is_ophys == True')
metadata_failures[['ophys_session_id','equipment_name','session_type','date_of_acquisition','is_ophys','metadata']]

,ophys_session_id,equipment_name,session_type,date_of_acquisition,is_ophys,metadata
3363,9.451241e+08,MESO.1,OPHYS_4_images_B,2019-09-12 15:42:50.871,1,0
3371,8.737206e+08,MESO.1,OPHYS_5_images_B_passive,2019-05-22 14:17:06.494,1,0
3407,9.932285e+08,MESO.1,OPHYS_1_images_G,2019-12-09 08:55:07.533,1,0
3408,1.006424e+09,MESO.1,OPHYS_1_images_G,2020-02-11 13:59:34.362,1,0
3436,1.010463e+09,MESO.1,OPHYS_4_images_H,2020-02-26 14:38:15.159,1,0
3443,9.940044e+08,MESO.1,OPHYS_5_images_H_passive,2019-12-12 15:05:06.596,1,0
3444,9.941366e+08,MESO.1,OPHYS_5_images_H_passive,2019-12-13 09:04:41.850,1,0
3446,1.011987e+09,MESO.1,OPHYS_5_images_H_passive,2020-03-03 15:10:21.773,1,0
3455,1.012619e+09,MESO.1,OPHYS_6_images_H,2020-03-05 12:21:43.514,1,0
3479,1.037717e+09,CAM2P.3,OPHYS_1_images_B,2020-07-21 11:34:58.508,1,0


,ophys_session_id,behavior_training_id,equipment_name,date_of_acquisition,donor_id,full_genotype,reporter_line,driver_line,sex,age_in_days,foraging_id,session_type,donor_in_ophys,project_id,project_code,in_filtered_experiments,average_projection,cell_specimen_table,corrected_fluorescence_traces,dff_traces,eye_tracking,licks,max_projection,metadata,motion_correction,ophys_timestamps,rewards,running_data_df,running_speed,segmentation_mask_image,stimulus_presentations,stimulus_templates,stimulus_timestamps,task_parameters,trials,is_ophys,timestamp
0,NaN,NaN,MESO.1,2019-04-01 13:24:11.225,810573033,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],M,120.0,e548098b-6be2-4591-b243-6807f4bf3521,OPHYS_0_images_A_habituation,True,589467762,MesoscopeDevelopment,True,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,1,1,NaN,1,1,1,1,1,0,2020-04-10 12:57:27.719853
1,NaN,NaN,MESO.1,2019-04-02 12:01:18.550,810573033,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],M,121.0,d249f644-f447-4456-8fad-df00053db2c7,OPHYS_0_images_A_habituation,True,589467762,MesoscopeDevelopment,True,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,1,1,NaN,1,1,1,1,1,0,2020-04-10 12:57:29.056899
2,NaN,NaN,MESO.1,2019-04-04 11:59:05.414,810573033,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],M,123.0,a0830bce-65ad-47ab-994c-1dbf95c84c5a,OPHYS_0_images_A_habituation,True,589467762,MesoscopeDevelopment,True,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,1,1,NaN,1,1,1,1,1,0,2020-10-30 02:17:45.115668
3,NaN,NaN,MESO.1,2019-04-08 08:38:56.310,813702144,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],M,118.0,0e1d251e-d065-4ef8-a5b7-fe9d5c3fa02d,OPHYS_0_images_A_habituation,True,589467762,MesoscopeDevelopment,True,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,1,1,NaN,1,1,1,1,1,0,2020-05-05 12:36:43.487980
4,NaN,NaN,MESO.1,2019-04-09 09:00:37.927,813702144,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],M,119.0,4d876357-c4d7-4277-80f7-8ac48a41fc00,OPHYS_0_images_A_habituation,True,589467762,MesoscopeDevelopment,True,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,1,1,1,NaN,1,1,1,1,1,0,2020-05-05 12:59:51.277083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3609,1.039232e+09,NaN,CAM2P.3,2020-07-29 09:39:54.458,1008591300,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",M,212.0,8dbb4233-ca4e-499b-a79b-309416623a92,OPHYS_6_images_A,True,873094517,VisualBehaviorTask1B,True,1.0,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1,1,1,1.0,1,1,1,1,1,1,2020-10-30 02:12:42.000985
3610,1.039247e+09,NaN,CAM2P.3,2020-07-29 11:15:06.127,1009380968,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",F,212.0,f19b645c-cd57-4541-a8c7-f2b704318bb9,OPHYS_6_images_A,True,873094517,VisualBehaviorTask1B,True,1.0,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1,1,1,1.0,1,1,1,1,1,1,2020-11-03 17:46:44.525957
3611,1.043719e+09,NaN,CAM2P.3,2020-08-17 14:09:54.159,1010925788,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",M,221.0,52f66f0a-6248-4c43-b81f-71c1ee6de3d5,OPHYS_6_images_A,True,873094517,VisualBehaviorTask1B,True,1.0,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1,1,1,1.0,1,1,1,1,1,1,2020-11-03 17:07:37.214932
3612,1.050242e+09,NaN,CAM2P.4,2020-09-14 10:49:49.338,1032508314,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",M,122.0,7c773653-c2a4-4506-9097-38fffc2a7e68,OPHYS_6_images_A,True,873094517,VisualBehaviorTask1B,True,1.0,1.0,1.0,1.0,1.0,1,1.0,1,1.0,1.0,1,1,1,1.0,1,1,1,1,1,1,2020-11-03 17:34:31.434594
